In [1]:
import pandas as pd
import pyodbc
import numpy as np
import os
import geopandas as gpd
import sqlalchemy as sal
from shapely import wkb
import re

In [2]:
path = r'T:\Tableau\tableauTitleVI\Datasources'

In [5]:
df = pd.read_csv(path + '\\TitleVIchangeovertime.csv')   
print(df)

     Year      Factor                   Universe  FactorTotal  MPOTotal  \
0    2007    Disabled  Non-Institutionalized Pop      33553.0  219933.0   
1    2007     Elderly                 Population      30468.0  237262.0   
2    2007    Minority                 Population      38828.0  237262.0   
3    2007     Poverty                 Households      16940.0   98311.0   
4    2007     Renters                 Households      42082.0   98311.0   
..    ...         ...                        ...          ...       ...   
100  2021    Minority                 Population      63125.0  268190.0   
101  2021     Poverty                 Households      15907.0  113975.0   
102  2021     Renters                 Households      51861.0  113975.0   
103  2021  Unemployed        Workforce (Pop 16+)      10128.0  144512.0   
104  2021   Zero Cars                 Households       9208.0  113975.0   

       MPOavg  
0    0.152560  
1    0.128415  
2    0.163650  
3    0.172310  
4    0.428050  
.. 

In [6]:
df.columns

Index(['Year', 'Factor', 'Universe', 'FactorTotal', 'MPOTotal', 'MPOavg'], dtype='object')

In [7]:
df.dtypes

Year             int64
Factor          object
Universe        object
FactorTotal    float64
MPOTotal       float64
MPOavg         float64
dtype: object

In [9]:
df.Universe.unique()

array(['Non-Institutionalized Pop', 'Population', 'Households',
       'Workforce (Pop 16+)'], dtype=object)

In [10]:
len('Non-Institutionalized Pop')

25

In [12]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [13]:
cursor.execute('''
            CREATE TABLE Title_VI_change_over_time(
            year int,
            factor varchar(50),
            universe varchar(50),
            factor_total float,
            mpo_total float,
            mpo_avg float
            )
               ''')

In [14]:
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Title_VI_change_over_time(year, factor, universe, factor_total, mpo_total, mpo_avg)
                VALUES (?,?,?,?,?,?)
                ''',
                   row.Year,
                   row.Factor,
                   row.Universe,
                   row.FactorTotal,
                   row.MPOTotal,
                   row.MPOavg
                  )
conn.commit()

In [3]:
gdf = gpd.read_file((path + '\\MPO_BG_TitleVI_Since2013.shp'))

In [5]:
gdf = gdf.drop(columns=['area_bg', 'area'])

In [6]:
constr = ("mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DGIS_CLMPO%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [7]:
engine = sal.create_engine(constr)
conn = engine.connect()

In [8]:
# define a function to convert to wkb 
def wkb_hexer(line):
    return line.wkb_hex

In [9]:
# apply to each row in the 'geometry' column
#gdf['geometry'] = gdf.geometry.apply(lambda x: x.wkt).apply(lambda x: re.sub('"(.*)"', '\\1', x))
gdf['geometry'] = gdf['geometry'].apply(wkb_hexer)

C:\ProgramData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [15]:
gdf.to_sql('MPO_Title_VI', con = engine, schema = 'dbo', index=False, if_exists = 'replace')

In [10]:
query = 'SELECT * FROM MPO_Title_VI'
engine.execute(query).fetchone()

(0, '410390001001', 0.001033544020098, 1.023208579897111, 0.223245508341188, 0.218181818181818, 0.143652322843074, 0.992401843168481, 0.142560829964901, 1.023208579897111, 0.0, 0.465094809044141, 0.824444444444444, 0.383444831456392, 0.029972776582845, 0.078167115902965, 0.047474747474747, 0.04857656894461, 0.398947991757863, 0.0, 0.0, 2.67, 1.023208579897111, 0.0, 0.0, 0.064079729246082, 0.0, 0.0, 0.0, 0.0, 0.16711590296496, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2013, '010300000001000000A000000011DE647CD7146AC1CB4AA72D2FE054418CD843AED7146AC1D2745B2F30E05441FAE492DCD7146AC15837B55831E05441818DB11CD8146AC1A518ACDE34E ... (4848 characters truncated) ... 46AC1793BA07C57E054417BAAD487DB146AC118FC92324BE054411669A50BD7146AC1A208E0832EE054412FCDA25BD7146AC1B045A1A32EE0544111DE647CD7146AC1CB4AA72D2FE05441')

In [11]:
def sql_data(table, server_name, database_name, driver, geodata = True): 
    constr= ("Driver={};"
             "Server={};"
             "Database={};"     
             "Trusted_Connection=yes;").format(driver,server_name, database_name)  
    conn = pyodbc.connect(constr)
    query = 'SELECT * FROM ' + table
    if geodata:
        data = gpd.read_postgis(query, conn, geom_col='geometry')
    else: 
        data = pd.read_sql(query, conn)
    conn.close()
    return data

In [12]:
# keys list has the function argument names 
keys = ['server_name', 'database_name', 'driver']
# values list holds the arguments for the SQL connection
values = ['rliddb.int.lcog.org,5433', 'GIS_CLMPO', '{SQL Server}']
# zip the keys and values into a dictionary 
serverargs = dict(zip(keys,values))

In [17]:
geodata = sql_data('MPO_Title_VI', **serverargs)

In [18]:
geodata.head(3)

,GEOID,PctInside,TotalPOP,PopEld,PctElderly,PctDisab,PopNInst5,PopNI5Disa,HHPop,GQPop,...,Disabled,Poor,Minority,UnEmp,LEP,HHzerocar,Renter,ComofConce,Year,geometry
0,410390001001,0.001034,1.023209,0.223246,0.218182,0.143652,0.992402,0.142561,1.023209,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2013,"POLYGON ((-13674171.887 5472444.713, -13674173..."
1,410390002001,0.003906,6.545767,1.894211,0.289379,0.128854,6.412728,0.826307,6.502805,0.042961,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2013,"MULTIPOLYGON (((-13697484.274 5483475.480, -13..."
2,410390003001,0.088736,82.524927,21.296755,0.258065,0.142791,79.467083,11.347177,82.524927,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2013,"MULTIPOLYGON (((-13697388.094 5483549.288, -13..."


In [19]:
geodata.shape

(1698, 41)

In [21]:
geodata[['GEOID', 'Year', 'geometry']].to_file(path+"\\BG_Geo.shp")